<a href="https://colab.research.google.com/github/Shahilasulthana/DeepLearning/blob/main/BP_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook presents a **manual implementation of backpropagation** in a neural network, developed to achieve a clear and in-depth understanding of how learning occurs within deep learning models.
The **Iris classification dataset** is used as the experimental dataset to demonstrate the complete learning process in a controlled and interpretable manner.

Instead of relying on **built-in backpropagation mechanisms** provided by high-level libraries such as **Keras or TensorFlow**, the neural network training process is implemented entirely **from scratch using NumPy**. This manual approach enables a transparent view of each step involved in forward and backward propagation.

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arshid/iris-flower-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'iris-flower-dataset' dataset.
Path to dataset files: /kaggle/input/iris-flower-dataset


In [3]:
path = kagglehub.dataset_download("arshid/iris-flower-dataset")
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'iris-flower-dataset' dataset.
Path to dataset files: /kaggle/input/iris-flower-dataset


In [4]:
import os

print("Dataset path:", path)
os.listdir(path)


Dataset path: /kaggle/input/iris-flower-dataset


['IRIS.csv']

In [5]:
import pandas as pd

file_path = os.path.join(path, "IRIS.csv")
df = pd.read_csv(file_path)
df.head()


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [6]:
import numpy as np


In [7]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


In [8]:
def sigmoid_derivative(a):
    return a * (1 - a)


In [9]:
X = df.drop("species", axis=1).values
y = df["species"].values

In [10]:
labels = np.unique(y)
label_map = {label: idx for idx, label in enumerate(labels)}

y_encoded = np.array([label_map[label] for label in y])


In [11]:
y_onehot = np.zeros((y_encoded.size, 3))
y_onehot[np.arange(y_encoded.size), y_encoded] = 1


In [12]:
X = (X - X.mean(axis=0)) / X.std(axis=0)


In [13]:
np.random.seed(42)

W1 = np.random.randn(4, 5)
b1 = np.zeros((1, 5))

W2 = np.random.randn(5, 3)
b2 = np.zeros((1, 3))


In [14]:
learning_rate = 0.1
epochs = 1000


In [15]:
for epoch in range(epochs):
    # Forward
    z1 = np.dot(X, W1) + b1
    a1 = sigmoid(z1)
    z2 = np.dot(a1, W2) + b2
    a2 = sigmoid(z2)

    # Loss
    loss = np.mean((y_onehot - a2) ** 2)

    # Backward
    delta_output = (a2 - y_onehot) * sigmoid_derivative(a2)
    delta_hidden = np.dot(delta_output, W2.T) * sigmoid_derivative(a1)

    # Update
    W2 -= learning_rate * np.dot(a1.T, delta_output)
    b2 -= learning_rate * np.sum(delta_output, axis=0, keepdims=True)
    W1 -= learning_rate * np.dot(X.T, delta_hidden)
    b1 -= learning_rate * np.sum(delta_hidden, axis=0, keepdims=True)

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")


Epoch 0, Loss: 0.3475
Epoch 100, Loss: 0.0126
Epoch 200, Loss: 0.0099
Epoch 300, Loss: 0.0092
Epoch 400, Loss: 0.0088
Epoch 500, Loss: 0.0087
Epoch 600, Loss: 0.0086
Epoch 700, Loss: 0.0085
Epoch 800, Loss: 0.0084
Epoch 900, Loss: 0.0083


In [16]:
predictions = np.argmax(a2, axis=1)
accuracy = np.mean(predictions == y_encoded)
print("Accuracy:", accuracy)


Accuracy: 0.9866666666666667


In [17]:
labels = np.unique(y)
label_map = {label: idx for idx, label in enumerate(labels)}
reverse_label_map = {idx: label for label, idx in label_map.items()}


In [18]:
def predict_iris(sample):
    """
    sample: list or array [sepal_length, sepal_width, petal_length, petal_width]
    """

    # Convert to numpy array
    sample = np.array(sample).reshape(1, -1)

    # Normalize using training data statistics
    sample = (sample - X.mean(axis=0)) / X.std(axis=0)

    # ---- Forward Propagation ----
    z1 = np.dot(sample, W1) + b1
    a1 = sigmoid(z1)

    z2 = np.dot(a1, W2) + b2
    a2 = sigmoid(z2)

    # Prediction
    predicted_class = np.argmax(a2)
    predicted_species = reverse_label_map[predicted_class]

    return predicted_species, a2


In [19]:
sample = [5.1, 3.5, 1.4, 0.2]
species, probabilities = predict_iris(sample)

print("Predicted Species:", species)
print("Class Probabilities:", probabilities)


Predicted Species: Iris-versicolor
Class Probabilities: [[0.00772823 0.99667814 0.00178728]]


In [20]:
sample = [6.0, 2.9, 4.5, 1.5]
species, probabilities = predict_iris(sample)

print("Predicted Species:", species)


Predicted Species: Iris-virginica
